# Tree Datasets

-- Trees reduce urban heat island effects and save lives! 
https://www.epa.gov/green-infrastructure/reduce-urban-heat-island-effect

From: https://www.nature.com/articles/s41597-023-02000-w#Sec8
## Canopy Height Model (CHM)
- Resolution: 1m spatial resolution:
- Values: The final CHM map has values ranging from 20 to 600 dm.
- Data Projection and Format for both CHM and Carbon Density: Both CHM and Carbon Density maps are in GeoTiff format under the NAD83 (National Spatial Reference System 2011) Universal Transverse Mercator Zone 18 North projection.
The CHM dataset was recorded in the unit of decimetre (dm) considering the precision of the LiDAR dataset (approximately 9 cm) and was saved in integer data format. Pixels with CHM values lower than 2 m were considered as non-tree, and masked out along with other non-vegetation land cover types. Pixels with CHM larger than 60 m were also masked out as non-trees to avoid misclassification with buildings since trees larger than 60 m were unlikely to exist in NYC. 

## Carbon density:
- Resolution: 1 m spatial
- Values: 0 - 410.1 ton/ha in floats
- Data Projection and Format: see above

A carbon density map over the tree canopy covered area for the entire NYC at 1 m spatial resolution in the unit of ton/ha. The Carbon density ranged from 0 to 410.1 ton/ha in float data format. 


## Individual Tree Dataset
- Refined polygons segmented from LiDAR-based CHM. 
- individual tree location by borough
- Attributes table of the tree crown polygons include the tree ID, polygon area (m2), tree top height (m), tree mean height (m), tree volume (m3), and carbon storage (predicted value, lower value, and upper value in ton).
- format: shapefiles

## Kernel Density Estimates of Trees

- Calculated in R tidy's st_kde() using the Individual Tree Dataset from above
- Scaled from -1 to 1


# Load Libraries

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.ma as ma
import pandas as pd
import rioxarray as rxr
import rasterio
from rasterio import plot as rioplot
from rasterio.plot import plotting_extent
import geopandas as gpd

import rasterstats as rs # zonalstatistics function used to extract raster values
import earthpy as et
import earthpy.plot as ep

import uuid


# Set consistent plotting style
sns.set_style("white")
sns.set(font_scale=1.5)

# Load Rasters and Extract by Point
- code reference: https://www.earthdatascience.org/courses/use-data-open-source-python/spatial-data-applications/lidar-remote-sensing-uncertainty/extract-data-from-raster/



In [2]:
# Load the CHM geotiff
file_name = '../data/greenness/NY_CHM_10Int260m.tif'

chm = rxr.open_rasterio(file_name, masked=True).squeeze()


# carbon density tiff
file_name = '../data/greenness/NY_CHM529_CCset0_Carbondensity.tif'

cd = rxr.open_rasterio(file_name, masked = True).squeeze()


# crown area tiff
file_name = '../data/L1/crown_area_ha.tiff'

ca = rxr.open_rasterio(file_name, masked = True).squeeze()

# tree density
file_name = '../data/L1/KDE_subsidized_1ha_unscaled.tiff'

td = rxr.open_rasterio(file_name, masked = True).squeeze()

In [ ]:
# View summary statistics of canopy height model
print('Mean CHM (dm):', chm.mean().values)
print('Max CHM (dm):', chm.max().values)
print('Min CHM (dm):', chm.min().values)



In [ ]:
# View summar statistics of the carbon density model
print('Mean Carbon Density:', cd.mean().values)
print('Max Carbon Density:', cd.max().values)
print('Min Carbon Density:', cd.min().values)



In [ ]:
# Plot histrogram of CHM


# Explore the data by plotting a histogram with earthpy
ax = ep.hist(chm.values,
             figsize=(8, 8),
             colors="green",
             xlabel="Lidar Estimated Tree Height (decimeter)",
             ylabel="Total Pixels",
             title="Distribution of Pixel Values \n Canopy Height Model")

# Turn off scientific notation
ax[1].ticklabel_format(useOffset=False,
                       style='plain')

In [ ]:
# Plot histrogram of carbon density


# Explore the data by plotting a histogram with earthpy
ax = ep.hist(cd.values,
             figsize=(8, 8),
             colors="green",
             xlabel="Carbon Density ton/ha",
             ylabel="Total Pixels",
             title="Distribution of Pixel Values \n Carbon Density Model")

# Turn off scientific notation
ax[1].ticklabel_format(useOffset=False,
                       style='plain')

In [3]:
# Load in affordable housing subsidy dataset as output from 01
subsidies = pd.read_csv('../data/L1/subsidies.csv')
print(subsidies.columns)
# drop unnamed: 0
subsidies = subsidies.drop(labels = 'Unnamed: 0', axis = 1)
subsidies["Value Per Unit"] = subsidies['assessed_value'] / subsidies['res_units']


Index(['Unnamed: 0', 'bbl', 'subsidy_program', 'end_date', 'standard_address',
       'city_id', 'city_name', 'boro_id', 'boro_name', 'cd_id', 'cd_name',
       'sba_id', 'sba_name', 'ccd_id', 'ccd_name', 'tract_10', 'res_units',
       'year_built', 'buildings', 'assessed_value', 'owner_name',
       'ser_violation', 'tax_delinquency', 'latitude', 'longitude',
       'subsidy_program_full', 'URL', 'Supply_Demand', 'Category', 'Scale',
       'Timeframe', 'Occupancy Tenure', 'Construction Type',
       'Max Income Restriction(%AMI)', 'Occupancy Demographic',
       'Income Designation', 'Project Eligibility', 'Developers/Owners',
       'geometry'],
      dtype='object')


In [4]:


subsidies["loc"] = subsidies['standard_address'] + ' ' + subsidies["city_name"]

subsidies = (subsidies[["loc", "bbl", "subsidy_program_full", "Value Per Unit", 
                        "boro_name", "latitude", "longitude", "Max Income Restriction(%AMI)",
                       "Occupancy Demographic", "Income Designation"]])




In [11]:
rentstab = pd.read_csv('../data/L1/rent_stabilized_clean.csv')
x = pd.concat([subsidies, rentstab])

# create unique id
x['row_id'] = [str(uuid.uuid4()) for _ in range(len(x.index))]

In [12]:
# Convert to spatial points
gdf = gpd.GeoDataFrame(x, geometry = gpd.points_from_xy(x['longitude'], x['latitude']), 
                                 crs = 4326)


type(gdf)

# Convert to the same projection as the canopy height and carbon density models: 
# NAD83 (National Spatial Reference System 2011) Universal Transverse Mercator Zone 18 North projection
# ESPG: https://epsg.io/6347 - 6347

gdf.to_crs(crs = 6347, inplace = True)

# ensure that is a points layer in the UTM projection
gdf.head()

,loc,bbl,subsidy_program_full,Value Per Unit,boro_name,latitude,longitude,Max Income Restriction(%AMI),Occupancy Demographic,Income Designation,row_id,geometry
0,80 Rutgers Slip New York City,1.002480e+09,Section 202/8,51766.513761,Manhattan,40.710802,-73.990193,50,Seniors; Persons with Disabilities,Very Low-income,413ac678-1129-48a8-9c80-a1102160ac58,POINT (585297.142 4507144.438)
1,80 Rutgers Slip New York City,1.002480e+09,420-c Tax Incentive Program,51766.513761,Manhattan,40.710802,-73.990193,60,Homeless; Persons with Disabilities; Families,Low-income,139d3af3-9d69-413d-869f-cfe785694af9,POINT (585297.142 4507144.438)
2,80 Rutgers Slip New York City,1.002480e+09,LIHTC 4%,51766.513761,Manhattan,40.710802,-73.990193,60,All Eligable,"Very Low-income, Low-income",0cabc93a-2138-4a1d-89d0-1c56c166c59e,POINT (585297.142 4507144.438)
3,15 Bialystoker Place New York City,1.003360e+09,Section 202/8,68058.984375,Manhattan,40.715819,-73.983302,50,Seniors; Persons with Disabilities,Very Low-income,65253d04-172f-42b8-bf2d-c7d515ca68ff,POINT (585872.753 4507708.039)
4,15 Bialystoker Place New York City,1.003360e+09,LIHTC 4%,68058.984375,Manhattan,40.715819,-73.983302,60,All Eligable,"Very Low-income, Low-income",0e6dd263-2e39-4479-b3bc-b1fd94574a5d,POINT (585872.753 4507708.039)


# Overlay points on top of raster data
A quick plot to check that subsidies actually overlay on top of the canopy height model. 
This is a good sanity check just to ensure data are in appropriate projection and location.

We have previously discussed the spatial extent of a raster. Here is where you will need to set the spatial extent when plotting raster using ep.plot_bands. If you do not specify a spatial extent, your raster will not line up properly with your geopandas object.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# We plot with the zeros in the data so the CHM can be better represented visually
ep.plot_bands(chm,
              extent=plotting_extent(chm,
                                     chm.rio.transform()),  # Set spatial extent
              cmap='Greys',
              title="NYC Canopy Height \n Subsidized Housing Locations",
              scale=False,
              ax=ax)
gdf.plot(ax=ax,
         marker='s',
         markersize=45,
         color='purple')
ax.set_axis_off()
plt.show()

# Buffer points and extract raster values
### Create buffer points with radius 500m (roughly equivalent to 1-2 city block)

In [13]:
# Create a buffered polygon layer from subsidized housing points
buffer = gdf[['row_id', 'geometry']]

# Buffer each point using a 500 meter circle radius
# and replace the point geometry with the new buffered geometry
buffer["geometry"] = gdf.geometry.buffer(500)
buffer.head()

# EXPORT to new shapefile


# Export the buffered point layer as a shapefile to use in zonal stats
subsidies_buffer_path = os.path.join('../data/L1/', 
                                "subsidies_buffer.shp")

buffer.to_file(subsidies_buffer_path)
print(len(buffer))

/Users/eihwood/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


53597


### Extract pixel values of CHM for each buffer zone (identified by unique BBL)

In [ ]:
np.nanmean(chm.values)

In [14]:
# Extract zonal stats from chm
extracted_chm = rs.zonal_stats(subsidies_buffer_path,
                                   chm.values,
                                   affine=chm.rio.transform(),
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="count min mean max median")

# View object type
type(extracted_chm)

/Users/eihwood/opt/anaconda3/lib/python3.9/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


list

In [15]:
# Turn extracted data into a pandas geodataframe
chm_df = gpd.GeoDataFrame.from_features(extracted_chm)



In [16]:
pd.set_option('display.max_columns', None)
chm_df.head()

chm_df = chm_df.rename(columns = {"min": "chm_min",
                      "max": "chm_max",
                      "mean": "chm_mean",
                      "median": "chm_median",
                        "count": "chm_count"})

In [18]:
len(chm_df)

53597

### Extract pixel values of Carbon Density for each buffer zone (identified by unique loc)

In [19]:
# Extract zonal stats from chm
extracted_cd = rs.zonal_stats(subsidies_buffer_path,
                                   cd.values,
                                   affine=cd.rio.transform(),
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="count min mean max median")

# View object type
type(extracted_cd)

list

In [20]:
# Turn extracted data into a pandas geodataframe
cd_df = gpd.GeoDataFrame.from_features(extracted_cd)



cd_df = cd_df.rename(columns = {"min": "cd_min",
                      "max": "cd_max",
                      "mean": "cd_mean",
                      "median": "cd_median",
                    "count": "cd_count"})

### Extract pixel values of Crown Area for each buffer zone (identified by unique loc)


In [21]:
# Extract zonal stats from chm
extracted_ca = rs.zonal_stats(subsidies_buffer_path,
                                   ca.values,
                                   affine=ca.rio.transform(),
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="count min mean max median")



In [22]:
# Turn extracted data into a pandas geodataframe
ca_df = gpd.GeoDataFrame.from_features(extracted_ca)



ca_df = ca_df.rename(columns = {"min": "ca_min",
                      "max": "ca_max",
                      "mean": "ca_mean",
                      "median": "ca_median",
                    "count": "ca_count"})

### Extract pixel values of Tree density for each buffer zone (identified by unique loc)


In [23]:
# Extract zonal stats from chm
extracted_td = rs.zonal_stats(subsidies_buffer_path,
                                   td.values,
                                   affine=td.rio.transform(),
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="min mean max median")

In [24]:
# Turn extracted data into a pandas geodataframe
td_df = gpd.GeoDataFrame.from_features(extracted_td)



td_df = td_df.rename(columns = {"min": "td_min",
                      "max": "td_max",
                      "mean": "td_mean",
                      "median": "td_median"})


In [25]:
# merge
cd_df = cd_df.drop(labels = 'geometry', axis = 1)
td_df = td_df.drop(labels = 'geometry', axis = 1)
ca_df = ca_df.drop(labels = 'geometry', axis = 1)



In [26]:
df_merge = pd.merge(chm_df, cd_df, left_on='row_id', right_on='row_id')
df_merge = pd.merge(df_merge, td_df, left_on='row_id', right_on='row_id')
df_merge = pd.merge(df_merge, ca_df, left_on='row_id', right_on='row_id')

df_merge.head()

,geometry,row_id,chm_min,chm_max,chm_mean,chm_count,chm_median,cd_min,cd_max,cd_mean,cd_count,cd_median,td_min,td_max,td_mean,td_median,ca_min,ca_max,ca_mean,ca_count,ca_median
0,"POLYGON ((585797.142 4507144.438, 585794.734 4...",413ac678-1129-48a8-9c80-a1102160ac58,21.0,593.0,132.516677,93753,134.0,0.0,409.264862,13.028841,784134,0.0,2.761813e-09,4.268787e-09,3.786921e-09,3.940875e-09,0.00,66.58,15.846250,80,7.090
1,"POLYGON ((585797.142 4507144.438, 585794.734 4...",139d3af3-9d69-413d-869f-cfe785694af9,21.0,593.0,132.516677,93753,134.0,0.0,409.264862,13.028841,784134,0.0,2.761813e-09,4.268787e-09,3.786921e-09,3.940875e-09,0.00,66.58,15.846250,80,7.090
2,"POLYGON ((585797.142 4507144.438, 585794.734 4...",0cabc93a-2138-4a1d-89d0-1c56c166c59e,21.0,593.0,132.516677,93753,134.0,0.0,409.264862,13.028841,784134,0.0,2.761813e-09,4.268787e-09,3.786921e-09,3.940875e-09,0.00,66.58,15.846250,80,7.090
3,"POLYGON ((586372.753 4507708.039, 586370.346 4...",65253d04-172f-42b8-bf2d-c7d515ca68ff,21.0,591.0,130.942872,161059,132.0,0.0,409.797943,22.097860,784143,0.0,6.843389e-10,3.386331e-09,1.933353e-09,1.878053e-09,1.94,64.92,28.787375,80,28.025
4,"POLYGON ((586372.753 4507708.039, 586370.346 4...",0e6dd263-2e39-4479-b3bc-b1fd94574a5d,21.0,591.0,130.942872,161059,132.0,0.0,409.797943,22.097860,784143,0.0,6.843389e-10,3.386331e-09,1.933353e-09,1.878053e-09,1.94,64.92,28.787375,80,28.025


In [27]:
# Merge back with subsidy data based on bbl

subsidy_with_green = pd.merge(x, df_merge, left_on = 'row_id', right_on = 'row_id')

In [29]:
subsidy_with_green.to_csv("../data/L1/subsidies_with_tree_data.csv")

In [28]:
len(subsidy_with_green)

53597